# The Sofa.Core.Node Class
The `Sofa.Core.Node` class is the fundamental building block of a SOFA scene graph. Every component (solvers, objects, forces) is attached to a `Node`, and nodes are organized in a tree or Directed Acyclic Graph (DAG).

### Learning Objectives
- Create and name nodes (including overloaded `addChild` and `addObject`).
- Use the **Context Manager** (`with` statement) for cleaner hierarchy building.
- Build scene hierarchies with **Prefabs** using the `add()` method.
- Access nodes and objects using dot notation, indexing, and specific search methods (`getChild`, `getObject`).
- Introspect the hierarchy (`getPathName`, `getRoot`, `getLinkPath`).
- Manipulate the graph (`moveChild`, `removeChild`, `detachFromGraph`).
- Inspect physics-related states (`getMass`, `getMechanicalState`, `computeEnergy`).
- Manage the node lifecycle (`init`, `isInitialized`) and the event system (`sendEvent`).


## 1. Environment Setup
First, we initialize the SOFA environment and import the necessary modules.

In [ ]:
import Sofa.Core
import SofaRuntime
SofaRuntime.init()

# Creating a root node
root = Sofa.Core.Node("root")
print(f"Created node: {root.name.value}")


## 2. Adding Children and Objects (Overloads)
Both `addChild` and `addObject` are overloaded to accept either a name string (to create a new one) or an existing instance.


In [ ]:
# Method 1: Create by name and pass extra parameters via kwargs
child1 = root.addChild("child1")
loop1 = child1.addObject("DefaultAnimationLoop", name="loop1")

# Method 2: Add an existing node instance
child2 = Sofa.Core.Node("child2")
root.addChild(child2)

print(f"Root children: {[c.name.value for c in root.children]}")


## 3. Advanced Scene Building
### A. Context Manager
`Sofa.Core.Node` supports the context manager protocol, allowing you to use the `with` statement. This makes the hierarchy structure clearer in code.


In [ ]:
with root.addChild("physics_node") as physics:
    physics.addObject("DefaultAnimationLoop", name="loop")
    # The 'physics' node is automatically initialized/entered here
    print(f"Inside 'with': {physics.getPathName()}")

print(f"Node added via context manager: {root.physics_node.name.value}")


## 4. Accessing and Searching
SOFA provides several ways to find nodes and objects.

### A. Dot Notation and Indexing
As seen in previous tutorials, you can use `node.childName` or `node["path.to.object"]`.


In [ ]:
# Dot notation
print(f"Access via dot: {root.child1.loop1.name.value}")

# Indexing (supports paths)
print(f"Access via path: {root['child1.loop1'].name.value}")


### B. Explicit Search Methods
Use `getChild`, `getObject`, and `hasObject` for more robust checks.


In [ ]:
if root.hasObject("physics_node.loop"):
    obj = root.getObject("physics_node.loop")
    print(f"Found object: {obj.name.value}")

child = root.getChild("child1")
print(f"Found child: {child.name.value}")


## 5. Hierarchy Manipulation
Nodes can be moved, removed, or detached from the graph.


In [ ]:
# moveChild(child, oldParent) moves a child to the current node (self)
child2.moveChild(child1, root) 
print(f"child1 parent is now: {[p.name.value for p in child1.parents]}")

# removeChild by name or instance
root.removeChild("physics_node")

# detachFromGraph removes the node from all its parents
child2.detachFromGraph()
print(f"child2 has {len(child2.parents)} parents after detach.")


## 6. Introspection
Methods to understand a node's position in the scene graph.


In [ ]:
node = root.addChild("introspection_test")
print(f"Path name: {node.getPathName()}")
print(f"Root node: {node.getRoot().name.value}")
print(f"Path to root: {node.getRootPath()}")
print(f"Link path: {node.getLinkPath()}")
print(f"Link parameter: {node.getAsACreateObjectParameter()}")


## 7. Physics and Simulation State
These methods provide access to physics-related information.


In [ ]:
# Note: These return relevant SOFA objects (MechanicalState, Mass, etc.) or None if not present.
SofaRuntime.importPlugin("Sofa.Component.StateContainer")
node.addObject("MechanicalObject", name="dofs")

print(f"Mechanical State: {node.getMechanicalState()}")
print(f"Mass: {node.getMass()}")
print(f"Mechanical Mapping: {node.getMechanicalMapping()}")
print(f"Energy (kinetic, potential): {node.computeEnergy()}")
print(f"Has ODE Solver: {node.hasODESolver()}")


## 8. Node Lifecycle and Events
### A. Initialization
Before simulation, nodes must be initialized.


In [ ]:
print(f"Is root initialized? {root.isInitialized()}")
root.init()
print(f"After init(): {root.isInitialized()}")


### B. Events
Events can be sent through the hierarchy.


In [ ]:
# sendEvent(data, eventName)
# Data can be any Python object.
node.sendEvent({"info": "start"}, "CustomEvent")
print("Event sent downstream.")


## 9. Properties: Children, Objects, and Parents
You can iterate over these collections using the `children`, `objects`, and `parents` properties.


In [ ]:
print(f"Root has {len(root.children)} children.")
for child in root.children:
    print(f" - Child: {child.name.value}")

for obj in root.objects:
    print(f" - Object: {obj.name.value}")


### Conclusion
The `Sofa.Core.Node` class provides a rich API for managing the simulation hierarchy, from basic scene building to advanced graph manipulation and introspection. Mastering these methods is key to creating complex and dynamic SOFA simulations.
